In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup

keyword = "다이슨"
last_num = 0
driver = webdriver.Chrome()
driver.get('https://cafe.naver.com/joonggonara')
time.sleep(3)

while True:
    # 두 번째 루프 이후 본문을 보던 iframe을 다시 검색 페이지로 복귀
    driver.switch_to.default_content()
    
    # 검색 키워드 입력 후 검색 클릭
    elem = driver.find_element(By.CLASS_NAME, "inp")
    elem.send_keys(keyword)
    elem.send_keys(Keys.RETURN)

    # iframe 변경
    iframe = driver.find_element(By.CSS_SELECTOR, ("iframe#cafe_main"))
    driver.switch_to.frame(iframe)

    # html = iframe의 html 코드를 모두 파싱
    html = BeautifulSoup(driver.page_source, 'html.parser')

    # 부모인 div class="article-board m-tcol-c">의 table > tbody > tr에 접근
    posts = html.select('div.article-board.m-tcol-c > table > tbody > tr')

    post_list = [ ]

    for post in posts: 
        post_dic = {'pnum': '', 'ptitle': '', 'pwriter': ''}
        post_dic['pnum'] = post.select('div.inner_number')[0].text
        post_dic['ptitle'] = post.select('a.article')[0].text.strip()
        post_dic['pwriter'] = post.select('a.m-tcol-c')[0].text

        post_list.append(post_dic)
        
    # 최초로 루프가 돌아간 시점
    if last_num == 0:
        last_num = post_list[0]['pnum']
        send_mail(999)

    # 두 번째 루프가 돌아간 이후
    else:
        for i in range(0, len(post_list)):
            # 이 조건문에 들어오는 건 5분 전 글보다 더 이전 글
            if (post_list[i]['pnum'] <= last_num):
                del post_list[i:]
                break

        # 새로운 글이 있다면 다시 리스트의 최신으로 넣어 줌
        if(len(post_list) != 0):
            last_num = post_list[0]['pnum']
            

    send_mail(len(post_list))
    print("총 " + str(len(post_list)) + "만큼의 새로운 글이 있습니다." + str(last_num))
    time.sleep(300)


총 15만큼의 새로운 글이 있습니다.954252271
총 0만큼의 새로운 글이 있습니다.954252271
총 0만큼의 새로운 글이 있습니다.954252271
총 0만큼의 새로운 글이 있습니다.954252271


KeyboardInterrupt: 

In [1]:
import smtplib

def send_mail(len):
    s = smtplib.SMTP('smtp.gmail.com', 587)
    s.starttls()
    s.login("gunny.algotogether@gmail.com", "gunnypas********")
    if(len == 999):
        message = ("검색 시작").encode('utf-8')
    else:
        message = ("새로운 상품이" + str(len) +  " 개 등록 되었습니다. ").encode('utf-8')
    s.sendmail("gunny.algotogether@gmail.com", "gunny.algotogether@gmail.com", message)
    s.quit()